# Modelo de asignación de flota considerando jornadas laborales de trabajadores y tiempos de descanso

In [1]:
import src.DataProcessing.read_data as rd
ruta = '../data/New_tables/LUNES_VIERNES_C4_C6.xlsx'
hoja_c4_c6 = 'C4_C6'
base = rd.tablas_ordinarios_c4_c6_v2(ruta,hoja_c4_c6)

base.head()

/Users/luis/Desktop/UNIVERSIDAD/PASANTIA_optimizacion/Researching_Model_Desing/src/DataProcessing/read_data.py:97: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  quitumbe_sur_norte['HORA_SALIDA'] = quitumbe_sur_norte['HORA SALIDA ǪUITUMBE'].apply(hora_a_minutos)
/Users/luis/Desktop/UNIVERSIDAD/PASANTIA_optimizacion/Researching_Model_Desing/src/DataProcessing/read_data.py:98: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  quitumbe_sur_norte.dropna(subset='ORD',inplace=True)
/Users/luis/Desktop/UNIVERSIDAD/PASANTIA_optimizacion/Researching_Model_Desing/src/DataProces

,ID,ORD,ACC.,COD. OP.,CIRCUITO,HORA SALIDA ǪUITUMBE,HORA LLEGADA ǪUITUMBE,TIEMPO EN ANDÉN,HORA_SALIDA,HORA_LLEGADA
0,1,1,>>,1Ǫ,C4,05:00,06:48,NaN,300.0,408.0
1,2,2,>>,2Ǫ,C4,05:08,06:56,NaN,308.0,416.0
2,3,3,>>,3Ǫ,C4,05:16,07:04,NaN,316.0,424.0
3,4,4,>>,4Ǫ,C4,05:24,07:12,NaN,324.0,432.0
4,5,5,>>,5Ǫ,C4,05:32,07:20,NaN,332.0,440.0


In [7]:
# Creamos los arcos
import pandas as pd
arcos_costo = []

arcos_costo_A1 = []
arcos_costo_A2 = []

# Parametros para controlar tiempito minimo y máximo de espera
l1 = 7
u1 = 15

# Parámetros para controlar tiempo mínimo y máximo de descanso
delta = 10 # Parametro de tolerancia
P = 30 # Parametro que indica longitud de pause
l2 = P
u2 = P + delta 

for i in base.iterrows():
    hora_llegada = i[1]['HORA_LLEGADA']
    
    base_filtrada_A1 = base[
        (l1<=base['HORA_SALIDA']-hora_llegada) &
        (base['HORA_SALIDA']-hora_llegada<=u1)]
    base_filtrada_A1 = pd.DataFrame({'ID':base_filtrada_A1['ID'],'HORA_SALIDA':(base_filtrada_A1['HORA_SALIDA'] - hora_llegada)}).reset_index()
    base_filtrada_A1['Arco-Costo A1'] = base_filtrada_A1.apply(lambda row: (row['ID'], row['HORA_SALIDA']), axis=1)
    arcos_costo_A1.append(base_filtrada_A1['Arco-Costo A1'].tolist())

    base_filtrada_A2 = base[
        (l2<=base['HORA_SALIDA']-hora_llegada) &
        (base['HORA_SALIDA']-hora_llegada<=u2)]
    base_filtrada_A2 = pd.DataFrame({'ID':base_filtrada_A2['ID'],'HORA_SALIDA':(base_filtrada_A2['HORA_SALIDA'] - hora_llegada)}).reset_index()
    base_filtrada_A2['Arco-Costo A2'] = base_filtrada_A2.apply(lambda row: (row['ID'], row['HORA_SALIDA']), axis=1)
    arcos_costo_A2.append(base_filtrada_A2['Arco-Costo A2'].tolist())
    
base['Arcos-Costo A1']=  pd.Series(arcos_costo_A1)
base['Arcos-Costo A2']=  pd.Series(arcos_costo_A2)
base['Costo-Inicial'] = 0

base.head()
    

,ID,ORD,ACC.,COD. OP.,CIRCUITO,HORA SALIDA ǪUITUMBE,HORA LLEGADA ǪUITUMBE,TIEMPO EN ANDÉN,HORA_SALIDA,HORA_LLEGADA,Arcos-Costo A1,Arcos-Costo A2,Costo-Inicial
0,1,1,>>,1Ǫ,C4,05:00,06:48,NaN,300.0,408.0,"[(16.0, 7.0), (17.0, 15.0), (105.0, 10.0)]","[(19.0, 33.0), (108.0, 34.0)]",0
1,2,2,>>,2Ǫ,C4,05:08,06:56,NaN,308.0,416.0,"[(17.0, 7.0), (18.0, 15.0), (106.0, 10.0)]","[(20.0, 35.0), (109.0, 38.0)]",0
2,3,3,>>,3Ǫ,C4,05:16,07:04,NaN,316.0,424.0,"[(18.0, 7.0), (107.0, 10.0)]","[(21.0, 37.0), (109.0, 30.0)]",0
3,4,4,>>,4Ǫ,C4,05:24,07:12,NaN,324.0,432.0,"[(19.0, 9.0), (108.0, 10.0)]","[(22.0, 39.0), (110.0, 34.0)]",0
4,5,5,>>,5Ǫ,C4,05:32,07:20,NaN,332.0,440.0,"[(20.0, 11.0), (109.0, 14.0)]","[(22.0, 31.0), (111.0, 38.0)]",0


In [8]:
import numpy as np

# Unidades
M = len(base['COD. OP.'].unique()) 

# Viajes 
N = base['ID'].unique()

# Creación del diccionario de costos
costos_A1 = {}
costos_A2 = {}
costos_inicio_fin = {}
for index,row in base.iterrows():
    costos_arcos_A1 = {(row['ID'],int(i[0])):i[1] for i in row['Arcos-Costo A1']}
    costos_A1.update(costos_arcos_A1)
    
    costos_arcos_A2 = {(row['ID'],int(i[0])):i[1] for i in row['Arcos-Costo A2']}
    costos_A2.update(costos_arcos_A2)
    
    costos_inicio_fin[(0,row['ID'])] = row['Costo-Inicial'] 
    costos_inicio_fin[(row['ID'],len(N)+1)] = 0
    
costos = costos_A1.copy()
costos.update(costos_A2)
costos.update(costos_inicio_fin)

arcos_A1 = costos_A1.keys()
arcos_A2 = costos_A2.keys()
arcos_unidades_A1 = []
arcos_unidades_A2 = []
arcos_unidades_inicio_fin = []

arcos_unidad_i_A1 = [(arco[0],arco[1]) for arco in arcos_A1 if arco[0]!=0 and arco[1]!=(len(N)+1)]
arcos_unidades_A1 += arcos_unidad_i_A1

arcos_unidad_i_A2 = [(arco[0],arco[1]) for arco in arcos_A2 if arco[0]!=0 and arco[1]!=(len(N)+1)]
arcos_unidades_A2 += arcos_unidad_i_A2

arcos_unidades_inicio_fin += [(0,row['ID']) for index,row in base.iterrows()]
arcos_unidades_inicio_fin += [(row['ID'],len(N)+1) for index,row in base.iterrows()]
    
print(len(arcos_unidades_A1))
print(len(arcos_unidades_A2))
print(len(arcos_unidades_inicio_fin))
arcos = arcos_unidades_A1+arcos_unidades_A2+arcos_unidades_inicio_fin

indices_A1 = list(set([elemento for tupla in arcos_unidades_A1 for elemento in tupla]))

print(len(indices_A1))

indices_A2 = list(set([elemento for tupla in arcos_unidades_A2 for elemento in tupla]))
print(len(indices_A2))

344
412
404
200
202


In [13]:
b = base['HORA_LLEGADA']
b = {key+1:value for key,value in b.to_dict().items()}
b

a = base['HORA_SALIDA']
a = {key+1:value for key,value in a.to_dict().items()}
a

{1: 300.0,
 2: 308.0,
 3: 316.0,
 4: 324.0,
 5: 332.0,
 6: 340.0,
 7: 348.0,
 8: 356.0,
 9: 364.0,
 10: 372.0,
 11: 379.0,
 12: 386.0,
 13: 393.0,
 14: 400.0,
 15: 407.0,
 16: 415.0,
 17: 423.0,
 18: 431.0,
 19: 441.0,
 20: 451.0,
 21: 461.0,
 22: 471.0,
 23: 481.0,
 24: 491.0,
 25: 501.0,
 26: 511.0,
 27: 521.0,
 28: 531.0,
 29: 541.0,
 30: 551.0,
 31: 561.0,
 32: 571.0,
 33: 581.0,
 34: 591.0,
 35: 601.0,
 36: 611.0,
 37: 621.0,
 38: 631.0,
 39: 641.0,
 40: 651.0,
 41: 661.0,
 42: 671.0,
 43: 681.0,
 44: 691.0,
 45: 698.0,
 46: 705.0,
 47: 712.0,
 48: 719.0,
 49: 726.0,
 50: 733.0,
 51: 740.0,
 52: 747.0,
 53: 754.0,
 54: 761.0,
 55: 771.0,
 56: 781.0,
 57: 791.0,
 58: 801.0,
 59: 811.0,
 60: 821.0,
 61: 829.0,
 62: 837.0,
 63: 845.0,
 64: 853.0,
 65: 861.0,
 66: 869.0,
 67: 877.0,
 68: 885.0,
 69: 893.0,
 70: 905.0,
 71: 917.0,
 72: 929.0,
 73: 941.0,
 74: 953.0,
 75: 965.0,
 76: 977.0,
 77: 987.0,
 78: 997.0,
 79: 1007.0,
 80: 1016.0,
 81: 1025.0,
 82: 1034.0,
 83: 1043.0,
 84: 105

In [12]:
import gurobipy as gp
from gurobipy import GRB

# Constantes M-mayúscula
M_T = 1440 # Minutos por día
M_P = 5 #

W = 1000 # Costo penalidad

m2 = gp.Model('Asignacion-Flota-V1')
# Terminar al alcanzar un Gap del 20%
m2.Params.MIPGap = 0.05
#m2.setParam('OutputFlag', 1)
# Enfocar estrategia en mejorar la cota dual
m2.Params.MIPFocus = 3

# Fijar generación muy agresiva de cortes
m2.Params.Cuts = 3

# Terminar luego de 180 segundos
m2.Params.TimeLimit = 14380

x = m2.addVars(arcos,vtype=GRB.BINARY,name='x')
z = m2.addVars(N,vtype=GRB.BINARY,name='z')

y0 = m2.addVars(N,vtype=GRB.BINARY,name='y1')
y1 = m2.addVars(N,vtype=GRB.BINARY,name='y2')
y2 = m2.addVars(N,vtype=GRB.BINARY,name='y3')

T = m2.addVars(N,vtype=GRB.INTEGER,name='T')
P = m2.addVars(N,vtype=GRB.INTEGER,name='P',lb=0,ub=2)

y = m2.addVars(N,vtype=GRB.BINARY,name='y')

m2.setObjective(gp.quicksum(costos[i,j]*x[i,j] for i,j in arcos) + W*z.sum('*'),GRB.MINIMIZE)

m2.addConstrs((x.sum(n,'*') == 1-z[n] for n in N), name=f"Constraints")
m2.addConstrs((x.sum('*', i) + x.sum(i, '*') == 0 for i in N), name=f'conservacion') 
m2.addConstr(x.sum(0,'*') <= M , name=f"Constraints_2")


m2.addConstrs((T[j] >= T[i] + (b[j]-b[i]) - M_T*(1-x[i,j]) for i,j in arcos_unidades_A1 ),name='T_upper_A1')
m2.addConstrs((T[j] <= T[i] + (b[j]-b[i]) + M_T*(1-x[i,j]) for i,j in arcos_unidades_A1 ),name='T_lower_A1')

m2.addConstrs((P[j] >= P[i] - M_P*(1-x[i,j]) for i,j in arcos_unidades_A1 ),name='P_upper_A1')
m2.addConstrs((P[j] <= P[i] + M_P*(1-x[i,j]) for i,j in arcos_unidades_A1 ),name='P_lower_A1')



m2.addConstrs((T[j] >= T[i] + (b[j]-b[i]) - M_T*(1-x[i,j]) for i,j in arcos_unidades_A2 ),name='T_upper_A2')
m2.addConstrs((T[j] <= T[i] + (b[j]-b[i]) + M_T*(1-x[i,j]) for i,j in arcos_unidades_A2 ),name='T_lower_A2')

m2.addConstrs((P[j] >= P[i] - M_P*(1-x[i,j]) for i,j in arcos_unidades_A2 ),name='P_upper_A2')
m2.addConstrs((P[j] <= P[i] + M_P*(1-x[i,j]) for i,j in arcos_unidades_A2 ),name='P_lower_A2')


m2.addConstrs((T[n] - 225 - delta <= M_T*(1-y0[n]) for n in N ),name='travel_1')
m2.addConstrs((T[n] - (b[n]-a[n])-225+delta-P>= - M_T*(1-y1[n]) for n in N ),name='travel_2')
m2.addConstrs((T[n]-675-P-2*delta <= (1-y1[n])*M_T for n in N ),name='travel_3')
m2.addConstrs((T[n]-(b[n]-a[n])-675-2*delta+2*P >= (1-y2[n])*M_T for n in N ),name='travel_4')

m2.addConstrs((y0[n]+y1[n]+y2[n]==1 for n in N ),name='travel_5')
m2.addConstrs((y1[n]+2*y2[n]==P[n] for n in N ),name='travel_5')

m2.write('Modelo.lp')






m2.optimize()

Set parameter MIPGap to value 0.05
Set parameter MIPFocus to value 3
Set parameter Cuts to value 3
Set parameter TimeLimit to value 14380
Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (mac64[arm] - Darwin 22.3.0 22D68)

CPU model: Apple M1 Max
Thread count: 10 physical cores, 10 logical processors, using up to 10 threads

Optimize a model with 3429 rows, 1968 columns and 12350 nonzeros
Model fingerprint: 0xcf2591b9
Variable types: 0 continuous, 1968 integer (1564 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+03]
  Objective range  [7e+00, 1e+03]
  Bounds range     [1e+00, 2e+00]
  RHS range        [1e+00, 2e+03]
Found heuristic solution: objective 202000.00000
Presolve removed 1921 rows and 1768 columns
Presolve time: 0.00s
Presolved: 1508 rows, 200 columns, 3016 nonzeros
Variable types: 0 continuous, 200 integer (0 binary)

Explored 0 nodes (0 simplex iterations) in 0.01 seconds (0.01 work units)
Thread count was 10 (of 10 available processors)

Solution count 1: 20